# Load Libraries

In [1]:
# import required libraries
import pandas as pd

# lets us display whole dataframe
pd.set_option('display.max_columns', None)

# Load Dataset

In [2]:
# import dataset
anz_data = pd.read_excel('ANZ synthesised transaction dataset.xlsx')
anz_data.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,35.39,2018-08-01,F,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,21.20,2018-08-01,F,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,5.71,2018-08-01,M,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,2117.22,2018-08-01,F,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,17.95,2018-08-01,F,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


In [3]:
anz_data.shape

(12043, 23)

# Data Preprocessing

In [4]:
# check duplicates
if anz_data['transaction_id'].unique().size == 12043:
    print("No Duplicate rows")

No Duplicate rows


In [5]:
# check missing values and unique values in each column
print("Number of missing values in each column \n", anz_data.isnull().sum())
print()
print("Number of unique values in each column \n", anz_data.nunique())

Number of missing values in each column 
 status                   0
card_present_flag     4326
bpay_biller_code     11158
account                  0
currency                 0
long_lat                 0
txn_description          0
merchant_id           4326
merchant_code        11160
first_name               0
balance                  0
date                     0
gender                   0
age                      0
merchant_suburb       4326
merchant_state        4326
extraction               0
amount                   0
transaction_id           0
country                  0
customer_id              0
merchant_long_lat     4326
movement                 0
dtype: int64

Number of unique values in each column 
 status                   2
card_present_flag        2
bpay_biller_code         3
account                100
currency                 1
long_lat               100
txn_description          6
merchant_id           5725
merchant_code            1
first_name              80
balance     

In [6]:
# remove columns which we won't use for our analysis
anz_data = anz_data.drop(columns=['bpay_biller_code','currency', 'merchant_code', 'first_name', 'transaction_id', 'country'])

In [7]:
# check unique values for 'status'
anz_data['status'].value_counts()

authorized    7717
posted        4326
Name: status, dtype: int64

In [8]:
# check unique values for 'card_present_flag'
anz_data['card_present_flag'].value_counts()

1.0    6194
0.0    1523
Name: card_present_flag, dtype: int64

In [9]:
# check what transactions uses card
anz_data[anz_data['card_present_flag'] == 1]['txn_description'].value_counts()

SALES-POS    3169
POS          3025
Name: txn_description, dtype: int64

In [10]:
# check what transactions doesn't use card
anz_data[anz_data['card_present_flag'] == 0]['txn_description'].value_counts()

SALES-POS    765
POS          758
Name: txn_description, dtype: int64

In [11]:
# check what transactions has missing 'card_present_flag'
anz_data[anz_data['card_present_flag'].isna()]['txn_description'].value_counts()

PAYMENT       2600
PAY/SALARY     883
INTER BANK     742
PHONE BANK     101
Name: txn_description, dtype: int64

The above type of transactions doesn't use card, it's the nature of transactions so filling missing values with 0.

In [12]:
# replacing missing values for card_present_flag with 0 and converting the column to int
anz_data['card_present_flag'] = anz_data['card_present_flag'].fillna(0).astype(int)

anz_data['card_present_flag'].value_counts()

1    6194
0    5849
Name: card_present_flag, dtype: int64

In [13]:
# make sure about one on one relationsip between account and customer_id
anz_data[['account', 'customer_id']].drop_duplicates().nunique()

account        100
customer_id    100
dtype: int64

In [14]:
# one of the column from above can be removed. so removing 'account' column
anz_data = anz_data.drop(columns=['account'])

In [15]:
# split long lat
anz_data[['customer_longitude', 'customer_latitude']] = anz_data['long_lat'].str.split(' ', expand = True)
anz_data[['merchant_longitude', 'merchant_latitude']] = anz_data['merchant_long_lat'].str.split(' ', expand = True)
anz_data = anz_data.drop(columns=['long_lat', 'merchant_long_lat'])

In [16]:
# check missing values
anz_data.isnull().sum()

status                   0
card_present_flag        0
txn_description          0
merchant_id           4326
balance                  0
date                     0
gender                   0
age                      0
merchant_suburb       4326
merchant_state        4326
extraction               0
amount                   0
customer_id              0
movement                 0
customer_longitude       0
customer_latitude        0
merchant_longitude    4326
merchant_latitude     4326
dtype: int64

Only merchant related information are missing, and this is because of the nature of transaction.

In [17]:
# change extraction to datetime datatypes
anz_data['extraction'] = pd.to_datetime(anz_data['extraction'])

In [18]:
# rename extraction column to transaction_date_time
anz_data = anz_data.rename(columns = {'extraction' : 'transaction_date_time'})

In [19]:
# print start date and end date in the dataset
print('Start Date: ', min(anz_data['transaction_date_time'].dt.date))
print('End Date: ', max(anz_data['transaction_date_time'].dt.date))

Start Date:  2018-08-01
End Date:  2018-10-31


In [20]:
# number of days
anz_data['transaction_date_time'].dt.date.nunique()

91

From 1st August 2018 till 31st October 2018, there should be data for 92 days but the dataset has transaction data for only 91 days. Transaction data for 1 day is missing.

# Save Dataset

In [21]:
# save cleaned dataset
anz_data.to_csv('C:/Users/grg2b/Downloads/ANZ - Virtual Experience Programs - Forage/ANZ_data_clean.csv', index=False)

The cleaned dataset is then saved as 'ANZ_data_clean.csv' and the EDA is performed using tableau, which can be found [**here**](https://public.tableau.com/app/profile/biswabal.gurung/viz/ANZ-VirtualExperiencePrograms-Forage-Task1/ANZTransactionDataAnalysis)